# Imports

In [1]:
import numpy as np
import itertools
import copy
import time
import networkx as nx

# Create Contingency Matrix

In [2]:
def generate_random_contingency_matrix(num_clusters_method_a, num_clusters_method_b, constant_sum):
    random_matrix = np.zeros((num_clusters_method_a, num_clusters_method_b), dtype=int)
    cluster_a_list = np.zeros((num_clusters_method_a), dtype=int)

    for row_index in range(constant_sum):
        random_index = np.random.randint(0, num_clusters_method_a)
        cluster_a_list[random_index] += 1

    for row_index in range(num_clusters_method_a):
        for _ in range(cluster_a_list[row_index]):
            random_index = np.random.randint(0, num_clusters_method_b)
            random_matrix[row_index, random_index] += 1

    return random_matrix


In [3]:
num_clusters_method_a = 4
num_clusters_method_b = 4
total_members = 100

contingency_matrix = generate_random_contingency_matrix(num_clusters_method_a, num_clusters_method_b, total_members)

print("Contingency Matrix:\n", contingency_matrix)

Contingency Matrix:
 [[10  5  2  8]
 [ 4 12  7  9]
 [ 8  5  5  5]
 [ 6  4  8  2]]


# Brute Force Matching

In [4]:
def brute_force_matching(matrix):
    num_clusters_a = matrix.shape[0]

    best_matching = None
    best_matching_sum = 0

    for permutation in list(itertools.permutations(range(num_clusters_a))):
        matching_sum = np.sum(matrix[np.arange(num_clusters_a), permutation])
        if matching_sum > best_matching_sum:
            best_matching_sum = matching_sum
            best_matching = permutation

    return best_matching, best_matching_sum

In [5]:
best_matching, best_matching_sum = brute_force_matching(contingency_matrix)
best_matching = [(index, number) for index, number in enumerate(best_matching)]
print("Best Matching: ", best_matching)
print("Best Matching Sum: ", best_matching_sum)

Best Matching:  [(0, 3), (1, 1), (2, 0), (3, 2)]
Best Matching Sum:  36


# Stable Matching Based Clustering Evaluation

In [6]:
class stable_matcher:

    def __init__(self, matrix):
        self.ready = False
        self.pairs = []
        self.matrix = np.zeros((matrix.shape[0], matrix.shape[1]), dtype=int)
        self.matrix = copy.deepcopy(matrix)

    def create_matrix(self, matrix):
        self.ready = False
        self.matrix = np.zeros((matrix.shape[0], matrix.shape[1]), dtype=int)
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                self.matrix[i, j] = matrix[i, j]

    def score(self):
        if self.ready:
            return np.sum(self.matrix[[pair[0] for pair in self.pairs], [pair[1] for pair in self.pairs]])
        return -1

    def find_pairs(self):

        def priorities(matrix):

            def rank(numbers):
                sorted_indices = np.argsort(numbers)[::-1]
                return sorted_indices

            output = []
            for comm1 in range(matrix.shape[0]):
                temp = []
                for comm2 in range(matrix.shape[1]):
                    common = matrix[comm1,comm2]
                    temp.append(common)
                output.append(rank(temp))
            return output

        def find_pair(List, value):
            for pair in List:
                if pair[1] == value:
                    return pair[0]

        def find_index_of_value_in_array(arr, target_value):
            indices = np.where(arr == target_value)[0]
            if indices.size > 0:
                return indices[0]
            else:
                return 

        self.ready = True
        self.pairs = []
        chooser = range(self.matrix.shape[0])
        wanted = range(self.matrix.shape[1])
        unmatched_chooser = [index for index in range(len(chooser))]
        unmatched_wanted = [index for index in range(len(wanted))]
        prior_chooser = priorities(self.matrix)
        prior_wanted = priorities(self.matrix.T)
        while(len(unmatched_chooser)):
            comm1 = unmatched_chooser.pop()
            for matching_pred in prior_chooser[comm1]:
                if matching_pred in unmatched_wanted:
                    self.pairs.append([comm1, matching_pred])
                    unmatched_wanted.remove(matching_pred)
                    break
                else:
                    matched_with_wanted = find_pair(self.pairs,matching_pred)
                    if find_index_of_value_in_array(prior_wanted[matching_pred], comm1) < find_index_of_value_in_array(prior_wanted[matching_pred], matched_with_wanted):
                        self.pairs.remove([matched_with_wanted, matching_pred])
                        self.pairs.append([comm1, matching_pred])
                        unmatched_chooser.append(matched_with_wanted)
                        break
        return self.pairs

In [7]:
stable = stable_matcher(contingency_matrix)
pairs = stable.find_pairs()
best_score =  stable.score()

print("Best Matching Based on Stable Matching: ", sorted(pairs, key=lambda x: x[0]))
print("Best Matching Sum Based on Stable Matching: ", best_score)

Best Matching Based on Stable Matching:  [[0, 0], [1, 1], [2, 3], [3, 2]]
Best Matching Sum Based on Stable Matching:  35


# Maximum Weighted Matching

In [8]:

class maximum_weighted_matching:

    def __init__(self, matrix):
        self.ready = False
        self.pairs = []
        self.num_clusters_method_a = matrix.shape[0]
        self.num_clusters_method_b = matrix.shape[1]
        self.graph = nx.Graph()
        self.graph.add_nodes_from([f'A{i}' for i in range(num_clusters_method_a)], bipartite=0)
        self.graph.add_nodes_from([f'B{i}' for i in range(num_clusters_method_b)], bipartite=1)
        self.RB_top = {n for n, d in self.graph.nodes(data=True) if d["bipartite"] == 0}
        for i in range(self.num_clusters_method_a):
            for j in range(self.num_clusters_method_b):
                if matrix[i][j] > 0:
                    self.graph.add_edge(f'A{i}', f'B{j}', weight=matrix[i][j])

    def create_biparticle_graph_from_contingency_matrix(self, matrix):
        self.ready = False
        self.pairs = []
        self.num_clusters_method_a = matrix.shape[0]
        self.num_clusters_method_b = matrix.shape[1]
        self.graph = nx.Graph()
        self.graph.add_nodes_from([f'A{i}' for i in range(num_clusters_method_a)], bipartite=0)
        self.graph.add_nodes_from([f'B{i}' for i in range(num_clusters_method_b)], bipartite=1)
        self.RB_top = {n for n, d in self.graph.nodes(data=True) if d["bipartite"] == 0}
        for i in range(self.num_clusters_method_a):
            for j in range(self.num_clusters_method_b):
                if matrix[i][j] > 0:
                    self.graph.add_edge(f'A{i}', f'B{j}', weight=matrix[i][j])

    def score(self):
        if self.ready:
            score = 0
            for i in self.pairs:
                score += self.graph[i[0]][i[1]]['weight'] # output of mximum matching in nx is transposed.
                # For example instead of (A0, B1) it returns (B0, A1). So I calculate score in this way to solve this issue.
            return score
        return -1

    def find_pairs(self):
        self.ready = True
        self.pairs = []
        matching = nx.max_weight_matching(self.graph)
        # matching = nx.algorithms.bipartite.maximum_matching(self.graph, self.RB_top)
        self.pairs = list(matching)
        return self.pairs

In [9]:
matcher = maximum_weighted_matching(contingency_matrix)
pairs = matcher.find_pairs()
best_score =  matcher.score()

print("Best Matching Based on Maximum Weighted Matching: ", sorted(pairs, key=lambda x: x[1]))
print("Best Matching Sum Based on Maximum Weighted Matching: ", best_score)

Best Matching Based on Maximum Weighted Matching:  [('B1', 'A1'), ('B0', 'A2'), ('A3', 'B2'), ('A0', 'B3')]
Best Matching Sum Based on Maximum Weighted Matching:  36


# Testing

In [12]:
def iterative_testing(num_clusters_method_a, num_clusters_method_b, total_members, num_iterations=100):
    # this function is used to test the performance of the stable matching algorithm
    # by running the algorithm multiple times and compare results with the maximum weighted matching.

    create_time_maximum_weighted_matching = []
    create_time_stable_matching = []

    run_time_maximum_weighted_matching = []
    run_time_stable_matching = []

    accuracy_stable_matching = []

    for _ in range(num_iterations):

        contingency_matrix = generate_random_contingency_matrix(num_clusters_method_a, num_clusters_method_b, total_members)

        stable = stable_matcher(contingency_matrix)

        start = time.time()
        stable.create_matrix(contingency_matrix)
        end = time.time()
        create_time_stable_matching.append(end-start)

        start = time.time()
        stable.find_pairs()
        end = time.time()
        run_time_stable_matching.append(end-start)

        max_matching = maximum_weighted_matching(contingency_matrix)

        start = time.time()
        max_matching.create_biparticle_graph_from_contingency_matrix(contingency_matrix)
        end = time.time()
        create_time_maximum_weighted_matching.append(end-start)

        start = time.time()
        max_matching.find_pairs()
        end = time.time()
        run_time_maximum_weighted_matching.append(end-start)

        accuracy_stable_matching.append(stable.score() / max_matching.score())

    print("Average Create Time Maximum Weighted Matching: ", np.mean(create_time_maximum_weighted_matching))
    print("Average Create Time Stable Matching: ", np.mean(create_time_stable_matching))
    print("Average Run Time Maximum Weighted Matching: ", np.mean(run_time_maximum_weighted_matching))
    print("Average Run Time Stable Matching: ", np.mean(run_time_stable_matching))
    print("Average Accuracy Stable Matching: ", np.mean(accuracy_stable_matching))
    

In [15]:
iterative_testing(num_clusters_method_a=15, num_clusters_method_b=14, total_members=1000, num_iterations=100)

Average Create Time Maximum Weighted Matching:  0.0002779960632324219
Average Create Time Stable Matching:  9.854793548583984e-05
Average Run Time Maximum Weighted Matching:  0.0038750147819519042
Average Run Time Stable Matching:  0.00039044618606567385
Average Accuracy Stable Matching:  0.9801954379736503


In [14]:
iterative_testing(num_clusters_method_a=100, num_clusters_method_b=100, total_members=10000, num_iterations=100)

Average Create Time Maximum Weighted Matching:  0.010745792388916016
Average Create Time Stable Matching:  0.0018314528465270996
Average Run Time Maximum Weighted Matching:  0.2871543526649475
Average Run Time Stable Matching:  0.00994124412536621
Average Accuracy Stable Matching:  0.964178047458783


In [16]:
iterative_testing(num_clusters_method_a=1000, num_clusters_method_b=1000, total_members=1000000, num_iterations=1)

Average Create Time Maximum Weighted Matching:  0.9968140125274658
Average Create Time Stable Matching:  0.14902377128601074
Average Run Time Maximum Weighted Matching:  161.64742159843445
Average Run Time Stable Matching:  0.5578997135162354
Average Accuracy Stable Matching:  0.974392523364486


In [10]:
num_clusters_method_a = 11
num_clusters_method_b = 11
total_members = 1000

contingency_matrix = generate_random_contingency_matrix(num_clusters_method_a, num_clusters_method_b, total_members)

In [11]:
start = time.time()
best_matching, best_matching_sum = brute_force_matching(contingency_matrix)
best_matching = [(index, number) for index, number in enumerate(best_matching)]
print("Best Matching: ", best_matching)
print("Best Matching Sum: ", best_matching_sum)
end = time.time()
print("BF Time Taken: ", end - start)

Best Matching:  [(0, 0), (1, 2), (2, 3), (3, 10), (4, 8), (5, 4), (6, 7), (7, 6), (8, 5), (9, 1), (10, 9)]
Best Matching Sum:  136
BF Time Taken:  233.98167395591736


In [19]:
start = time.time()
pairs = stable_matcher(contingency_matrix)
sorted_pairs = sorted(pairs, key=lambda x: x[0])
best_score =  np.sum(contingency_matrix[[pair[0] for pair in sorted_pairs], [pair[1] for pair in sorted_pairs]])
print("Best Matching Based on Stable Matching: ", sorted_pairs)
print("Best Matching Sum Based on Stable Matching: ", best_score)
end = time.time()
print(f"SM Time Taken: {(end - start):.2f}")

Best Matching Based on Stable Matching:  [[0, 0], [1, 2], [2, 10], [3, 1], [4, 8], [5, 4], [6, 7], [7, 6], [8, 5], [9, 3], [10, 9]]
Best Matching Sum Based on Stable Matching:  135
SM Time Taken: 0.00


In [16]:
print(f'Accuracy: {best_score/ best_matching_sum}')

Accuracy: 0.9926470588235294


In [20]:
num_clusters_method_a = 10
num_clusters_method_b = 10
total_members = 1000

contingency_matrix = generate_random_contingency_matrix(num_clusters_method_a, num_clusters_method_b, total_members)

start = time.time()
best_matching, best_matching_sum = brute_force_matching(contingency_matrix)
best_matching = [(index, number) for index, number in enumerate(best_matching)]
print("Best Matching: ", best_matching)
print("Best Matching Sum: ", best_matching_sum)
end = time.time()
print("BF Time Taken: ", end - start)

start = time.time()
pairs = stable_matcher(contingency_matrix)
sorted_pairs = sorted(pairs, key=lambda x: x[0])
best_score =  np.sum(contingency_matrix[[pair[0] for pair in sorted_pairs], [pair[1] for pair in sorted_pairs]])
print("Best Matching Based on Stable Matching: ", sorted_pairs)
print("Best Matching Sum Based on Stable Matching: ", best_score)
end = time.time()
print(f"SM Time Taken: {(end - start):.2f}")

print(f'Accuracy: {best_score/ best_matching_sum}')

Best Matching:  [(0, 4), (1, 0), (2, 8), (3, 3), (4, 1), (5, 6), (6, 9), (7, 2), (8, 7), (9, 5)]
Best Matching Sum:  148
BF Time Taken:  18.0100576877594
Best Matching Based on Stable Matching:  [[0, 4], [1, 0], [2, 2], [3, 8], [4, 1], [5, 6], [6, 9], [7, 3], [8, 7], [9, 5]]
Best Matching Sum Based on Stable Matching:  147
SM Time Taken: 0.00
Accuracy: 0.9932432432432432


In [23]:
num_clusters_method_a = 15
num_clusters_method_b = 15
total_members = 10000

contingency_matrix = generate_random_contingency_matrix(num_clusters_method_a, num_clusters_method_b, total_members)

start = time.time()
pairs = stable_matcher(contingency_matrix)
sorted_pairs = sorted(pairs, key=lambda x: x[0])
best_score =  np.sum(contingency_matrix[[pair[0] for pair in sorted_pairs], [pair[1] for pair in sorted_pairs]])
print("Best Matching Based on Stable Matching: ", sorted_pairs)
print("Best Matching Sum Based on Stable Matching: ", best_score)
end = time.time()
print(f"SM Time Taken: {(end - start):.2f}")

Best Matching Based on Stable Matching:  [[0, 0], [1, 13], [2, 12], [3, 11], [4, 8], [5, 6], [6, 7], [7, 3], [8, 9], [9, 5], [10, 2], [11, 14], [12, 10], [13, 4], [14, 1]]
Best Matching Sum Based on Stable Matching:  836
SM Time Taken: 0.00


In [25]:
num_clusters_method_a = 50
num_clusters_method_b = 50
total_members = 100000

contingency_matrix = generate_random_contingency_matrix(num_clusters_method_a, num_clusters_method_b, total_members)

start = time.time()
pairs = stable_matcher(contingency_matrix)
sorted_pairs = sorted(pairs, key=lambda x: x[0])
best_score =  np.sum(contingency_matrix[[pair[0] for pair in sorted_pairs], [pair[1] for pair in sorted_pairs]])
print("Best Matching Based on Stable Matching: ", sorted_pairs)
print("Best Matching Sum Based on Stable Matching: ", best_score)
end = time.time()
print(f"SM Time Taken: {(end - start):.2f}")

Best Matching Based on Stable Matching:  [[0, 7], [1, 26], [2, 38], [3, 13], [4, 27], [5, 18], [6, 10], [7, 36], [8, 47], [9, 20], [10, 19], [11, 41], [12, 28], [13, 23], [14, 24], [15, 2], [16, 29], [17, 0], [18, 45], [19, 4], [20, 9], [21, 16], [22, 46], [23, 3], [24, 5], [25, 25], [26, 31], [27, 21], [28, 44], [29, 14], [30, 34], [31, 6], [32, 22], [33, 11], [34, 8], [35, 32], [36, 17], [37, 40], [38, 12], [39, 15], [40, 35], [41, 37], [42, 43], [43, 42], [44, 39], [45, 33], [46, 49], [47, 48], [48, 30], [49, 1]]
Best Matching Sum Based on Stable Matching:  2624
SM Time Taken: 0.00


In [28]:
num_clusters_method_a = 100
num_clusters_method_b = 100
total_members = 1000000

contingency_matrix = generate_random_contingency_matrix(num_clusters_method_a, num_clusters_method_b, total_members)

start = time.time()
pairs = stable_matcher(contingency_matrix)
sorted_pairs = sorted(pairs, key=lambda x: x[0])
best_score =  np.sum(contingency_matrix[[pair[0] for pair in sorted_pairs], [pair[1] for pair in sorted_pairs]])
print("Best Matching Based on Stable Matching: ", sorted_pairs)
print("Best Matching Sum Based on Stable Matching: ", best_score)
end = time.time()
print(f"SM Time Taken: {(end - start):.2f}")

Best Matching Based on Stable Matching:  [[0, 17], [1, 63], [2, 58], [3, 32], [4, 33], [5, 30], [6, 66], [7, 43], [8, 93], [9, 53], [10, 83], [11, 77], [12, 56], [13, 11], [14, 20], [15, 48], [16, 8], [17, 55], [18, 27], [19, 82], [20, 10], [21, 50], [22, 0], [23, 52], [24, 45], [25, 67], [26, 14], [27, 7], [28, 68], [29, 80], [30, 49], [31, 1], [32, 65], [33, 86], [34, 25], [35, 59], [36, 79], [37, 72], [38, 92], [39, 75], [40, 9], [41, 70], [42, 13], [43, 42], [44, 71], [45, 97], [46, 98], [47, 19], [48, 95], [49, 5], [50, 54], [51, 26], [52, 46], [53, 34], [54, 47], [55, 51], [56, 12], [57, 62], [58, 85], [59, 38], [60, 29], [61, 60], [62, 22], [63, 91], [64, 35], [65, 18], [66, 84], [67, 21], [68, 28], [69, 4], [70, 40], [71, 88], [72, 89], [73, 44], [74, 61], [75, 2], [76, 39], [77, 24], [78, 6], [79, 69], [80, 57], [81, 96], [82, 36], [83, 78], [84, 90], [85, 3], [86, 64], [87, 37], [88, 74], [89, 16], [90, 15], [91, 81], [92, 23], [93, 94], [94, 73], [95, 99], [96, 76], [97, 31]

In [27]:
num_clusters_method_a = 1000
num_clusters_method_b = 1000
total_members = 10000000

contingency_matrix = generate_random_contingency_matrix(num_clusters_method_a, num_clusters_method_b, total_members)

start = time.time()
pairs = stable_matcher(contingency_matrix)
sorted_pairs = sorted(pairs, key=lambda x: x[0])
best_score =  np.sum(contingency_matrix[[pair[0] for pair in sorted_pairs], [pair[1] for pair in sorted_pairs]])
print("Best Matching Based on Stable Matching: ", sorted_pairs)
print("Best Matching Sum Based on Stable Matching: ", best_score)
end = time.time()
print(f"SM Time Taken: {(end - start):.2f}")

Best Matching Based on Stable Matching:  [[0, 720], [1, 855], [2, 575], [3, 491], [4, 807], [5, 372], [6, 469], [7, 909], [8, 985], [9, 980], [10, 362], [11, 181], [12, 717], [13, 929], [14, 233], [15, 395], [16, 99], [17, 81], [18, 893], [19, 219], [20, 844], [21, 611], [22, 267], [23, 887], [24, 70], [25, 509], [26, 535], [27, 521], [28, 327], [29, 872], [30, 66], [31, 832], [32, 916], [33, 448], [34, 253], [35, 111], [36, 776], [37, 180], [38, 24], [39, 605], [40, 747], [41, 258], [42, 260], [43, 962], [44, 958], [45, 567], [46, 283], [47, 546], [48, 360], [49, 782], [50, 606], [51, 214], [52, 409], [53, 904], [54, 759], [55, 197], [56, 737], [57, 620], [58, 464], [59, 419], [60, 780], [61, 924], [62, 968], [63, 865], [64, 994], [65, 35], [66, 217], [67, 4], [68, 11], [69, 266], [70, 158], [71, 177], [72, 137], [73, 617], [74, 207], [75, 710], [76, 206], [77, 615], [78, 79], [79, 629], [80, 764], [81, 321], [82, 240], [83, 308], [84, 61], [85, 145], [86, 964], [87, 911], [88, 804], 